In [63]:
# Setup Connection to MySQL (Sakila)

import pandas as pd
import mysql.connector

conn = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'port': 8889,
    'database': 'sakila',  # <--- This needs to match your actual DB name
    'raise_on_warnings': True
}


In [65]:
# The below method establishes the session with MySQL server 
mydb = mysql.connector.connect(**conn)

In [67]:
# Cursor object executes the SQL statements and interact with the MySQL server using a MySQLConnection object
cursor = mydb.cursor(dictionary=True)

# CHAPTER 5

In [47]:
# Adding Two Columns in payment Table
# We can add amount (payment amount) with a hypothetical service_fee of 2.00 (just for the math example). This allows you to validate that the final column is correctly computed.

query = """
SELECT payment_id,
       amount,
       amount + 2 AS amount_with_service_fee
FROM payment
LIMIT 10;
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(f"Payment ID: {row['payment_id']}, Original Amount: ${row['amount']:.2f}, Amount with Service Fee: ${row['amount_with_service_fee']:.2f}")



Payment ID: 1, Original Amount: $2.99, Amount with Service Fee: $4.99
Payment ID: 2, Original Amount: $0.99, Amount with Service Fee: $2.99
Payment ID: 3, Original Amount: $5.99, Amount with Service Fee: $7.99
Payment ID: 4, Original Amount: $0.99, Amount with Service Fee: $2.99
Payment ID: 5, Original Amount: $9.99, Amount with Service Fee: $11.99
Payment ID: 6, Original Amount: $4.99, Amount with Service Fee: $6.99
Payment ID: 7, Original Amount: $4.99, Amount with Service Fee: $6.99
Payment ID: 8, Original Amount: $0.99, Amount with Service Fee: $2.99
Payment ID: 9, Original Amount: $3.99, Amount with Service Fee: $5.99
Payment ID: 10, Original Amount: $5.99, Amount with Service Fee: $7.99


In [51]:
# calculating percentage and Tracking Percent Change

# percentage of each payment amount compared to the average payment amount across all rows. This can help you analyze how much each payment deviates from the average — useful for detecting large or small transactions relative to the typical payment.


# Step 1 - Calculate Average Payment (needed for the percentage calculation)
query = """
SELECT AVG(amount) AS avg_payment FROM payment
"""
cursor.execute(query)
avg_payment = cursor.fetchone()['avg_payment']

# Step 2 - Fetch Payment Amounts and Calculate Percentage of Average for Each
query = """
SELECT payment_id, amount
FROM payment
LIMIT 10
"""
cursor.execute(query)

print("\nPayment Amount as Percentage of Average Payment:")
for row in cursor.fetchall():
    percentage_of_avg = (row['amount'] / avg_payment) * 100
    print(f"Payment ID: {row['payment_id']}, Amount: ${row['amount']:.2f}, Percentage of Average: {percentage_of_avg:.2f}%")


Payment Amount as Percentage of Average Payment:
Payment ID: 1, Amount: $2.99, Percentage of Average: 71.17%
Payment ID: 2, Amount: $0.99, Percentage of Average: 23.56%
Payment ID: 3, Amount: $5.99, Percentage of Average: 142.57%
Payment ID: 4, Amount: $0.99, Percentage of Average: 23.56%
Payment ID: 5, Amount: $9.99, Percentage of Average: 237.78%
Payment ID: 6, Amount: $4.99, Percentage of Average: 118.77%
Payment ID: 7, Amount: $4.99, Percentage of Average: 118.77%
Payment ID: 8, Amount: $0.99, Percentage of Average: 23.56%
Payment ID: 9, Amount: $3.99, Percentage of Average: 94.97%
Payment ID: 10, Amount: $5.99, Percentage of Average: 142.57%


In [53]:

# Aggregate Functions - Total and Average Payment Amount
query = """
SELECT 
    SUM(amount) AS total_revenue,
    ROUND(AVG(amount), 2) AS avg_payment
FROM payment
"""
cursor.execute(query)
agg_result = cursor.fetchone()

print(f"Total Revenue: ${agg_result['total_revenue']:.2f}")
print(f"Average Payment: ${agg_result['avg_payment']:.2f}")

# Finding the Median Payment (manual since MySQL lacks direct MEDIAN())
query = """
SELECT amount
FROM payment
ORDER BY amount
"""
cursor.execute(query)

amounts = [row['amount'] for row in cursor.fetchall()]

median_amount = amounts[len(amounts) // 2]
print(f"\nMedian Payment: ${median_amount:.2f}")

# Finding the Mode (most frequent payment amount)
query = """
SELECT amount, COUNT(*) AS frequency
FROM payment
GROUP BY amount
ORDER BY frequency DESC
LIMIT 1
"""
cursor.execute(query)
mode_result = cursor.fetchone()

print(f"\nMode Payment Amount: ${mode_result['amount']:.2f}, Frequency: {mode_result['frequency']} times")


Total Revenue: $67406.56
Average Payment: $4.20

Median Payment: $3.99

Mode Payment Amount: $4.99, Frequency: 3789 times


In [69]:

# 1️⃣ Fetch All Payment Amounts (ordered) - required for median/quantiles
query = """
SELECT amount
FROM payment
ORDER BY amount
"""
cursor.execute(query)
amounts = [row['amount'] for row in cursor.fetchall()]

# 2️⃣ Define Python Function to Calculate Median
def calculate_median(values):
    n = len(values)
    if n % 2 == 1:
        return values[n // 2]
    else:
        return (values[n // 2 - 1] + values[n // 2]) / 2

median_amount = calculate_median(amounts)
print(f"\nMedian Payment Amount (via function): ${median_amount:.2f}")

# 3️⃣ Define Python Function to Calculate Any Quantile
def calculate_quantile(values, quantile):
    index = int(len(values) * quantile)
    return values[index]

# 4️⃣ Calculate Quartiles (25%, 50%, 75%)
q1 = calculate_quantile(amounts, 0.25)
q2 = median_amount  # This is the 50th percentile
q3 = calculate_quantile(amounts, 0.75)

print("\nQuartiles:")
print(f"25th Percentile (Q1): ${q1:.2f}")
print(f"50th Percentile (Median, Q2): ${q2:.2f}")
print(f"75th Percentile (Q3): ${q3:.2f}")

# 5️⃣ Calculate Additional Percentiles (10%, 90%)
p10 = calculate_quantile(amounts, 0.10)
p90 = calculate_quantile(amounts, 0.90)

print("\nExtra Percentiles:")
print(f"10th Percentile: ${p10:.2f}")
print(f"90th Percentile: ${p90:.2f}")



print("\n✅ Median function + quantiles calculated successfully!")



Median Payment Amount (via function): $3.99

Quartiles:
25th Percentile (Q1): $2.99
50th Percentile (Median, Q2): $3.99
75th Percentile (Q3): $4.99

Extra Percentiles:
10th Percentile: $0.99
90th Percentile: $6.99

✅ Median function + quantiles calculated successfully!


# CHAPTER 6

This chapter is all about JOINs, table relationships, and performing math across joined tables — so we’ll adapt all of that for the Sakila database.

In [83]:
# Basic Inner Join (Rental + Customer)
query = """
SELECT rental.rental_id, customer.first_name, customer.last_name, rental.rental_date
FROM rental
JOIN customer ON rental.customer_id = customer.customer_id
LIMIT 10;
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(row)


{'rental_id': 76, 'first_name': 'MARY', 'last_name': 'SMITH', 'rental_date': datetime.datetime(2005, 5, 25, 11, 30, 37)}
{'rental_id': 573, 'first_name': 'MARY', 'last_name': 'SMITH', 'rental_date': datetime.datetime(2005, 5, 28, 10, 35, 23)}
{'rental_id': 1185, 'first_name': 'MARY', 'last_name': 'SMITH', 'rental_date': datetime.datetime(2005, 6, 15, 0, 54, 12)}
{'rental_id': 1422, 'first_name': 'MARY', 'last_name': 'SMITH', 'rental_date': datetime.datetime(2005, 6, 15, 18, 2, 53)}
{'rental_id': 1476, 'first_name': 'MARY', 'last_name': 'SMITH', 'rental_date': datetime.datetime(2005, 6, 15, 21, 8, 46)}
{'rental_id': 1725, 'first_name': 'MARY', 'last_name': 'SMITH', 'rental_date': datetime.datetime(2005, 6, 16, 15, 18, 57)}
{'rental_id': 2308, 'first_name': 'MARY', 'last_name': 'SMITH', 'rental_date': datetime.datetime(2005, 6, 18, 8, 41, 48)}
{'rental_id': 2363, 'first_name': 'MARY', 'last_name': 'SMITH', 'rental_date': datetime.datetime(2005, 6, 18, 13, 33, 59)}
{'rental_id': 3284, 'fi

In [81]:
# Left Join (All Rentals + Payment Info if Exists)
query = """
SELECT rental.rental_id, customer.first_name, payment.amount
FROM rental
LEFT JOIN payment ON rental.rental_id = payment.rental_id
JOIN customer ON rental.customer_id = customer.customer_id
LIMIT 10;
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(row)


{'rental_id': 76, 'first_name': 'MARY', 'amount': Decimal('2.99')}
{'rental_id': 573, 'first_name': 'MARY', 'amount': Decimal('0.99')}
{'rental_id': 1185, 'first_name': 'MARY', 'amount': Decimal('5.99')}
{'rental_id': 1422, 'first_name': 'MARY', 'amount': Decimal('0.99')}
{'rental_id': 1476, 'first_name': 'MARY', 'amount': Decimal('9.99')}
{'rental_id': 1725, 'first_name': 'MARY', 'amount': Decimal('4.99')}
{'rental_id': 2308, 'first_name': 'MARY', 'amount': Decimal('4.99')}
{'rental_id': 2363, 'first_name': 'MARY', 'amount': Decimal('0.99')}
{'rental_id': 3284, 'first_name': 'MARY', 'amount': Decimal('3.99')}
{'rental_id': 4526, 'first_name': 'MARY', 'amount': Decimal('5.99')}


In [87]:
 # Right Join Equivalent (MySQL Note)
query = """
SELECT payment.payment_id, rental.rental_date, payment.amount
FROM payment
RIGHT JOIN rental ON rental.rental_id = payment.rental_id
LIMIT 10;
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(row)


{'payment_id': 3504, 'rental_date': datetime.datetime(2005, 5, 24, 22, 53, 30), 'amount': Decimal('2.99')}
{'payment_id': 12377, 'rental_date': datetime.datetime(2005, 5, 24, 22, 54, 33), 'amount': Decimal('2.99')}
{'payment_id': 11032, 'rental_date': datetime.datetime(2005, 5, 24, 23, 3, 39), 'amount': Decimal('3.99')}
{'payment_id': 8987, 'rental_date': datetime.datetime(2005, 5, 24, 23, 4, 41), 'amount': Decimal('4.99')}
{'payment_id': 6003, 'rental_date': datetime.datetime(2005, 5, 24, 23, 5, 21), 'amount': Decimal('6.99')}
{'payment_id': 14728, 'rental_date': datetime.datetime(2005, 5, 24, 23, 8, 7), 'amount': Decimal('0.99')}
{'payment_id': 7274, 'rental_date': datetime.datetime(2005, 5, 24, 23, 11, 53), 'amount': Decimal('1.99')}
{'payment_id': 6440, 'rental_date': datetime.datetime(2005, 5, 24, 23, 31, 46), 'amount': Decimal('4.99')}
{'payment_id': 3386, 'rental_date': datetime.datetime(2005, 5, 25, 0, 0, 40), 'amount': Decimal('4.99')}
{'payment_id': 10785, 'rental_date': date

In [89]:
# Full Outer Join Equivalent (MySQL Workaround)
query = """
SELECT rental.rental_id, customer.first_name, customer.last_name, payment.amount
FROM rental
LEFT JOIN payment ON rental.rental_id = payment.rental_id
JOIN customer ON rental.customer_id = customer.customer_id

UNION

SELECT rental.rental_id, customer.first_name, customer.last_name, payment.amount
FROM payment
RIGHT JOIN rental ON rental.rental_id = payment.rental_id
JOIN customer ON rental.customer_id = customer.customer_id
LIMIT 10;
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(row)


{'rental_id': 76, 'first_name': 'MARY', 'last_name': 'SMITH', 'amount': Decimal('2.99')}
{'rental_id': 573, 'first_name': 'MARY', 'last_name': 'SMITH', 'amount': Decimal('0.99')}
{'rental_id': 1185, 'first_name': 'MARY', 'last_name': 'SMITH', 'amount': Decimal('5.99')}
{'rental_id': 1422, 'first_name': 'MARY', 'last_name': 'SMITH', 'amount': Decimal('0.99')}
{'rental_id': 1476, 'first_name': 'MARY', 'last_name': 'SMITH', 'amount': Decimal('9.99')}
{'rental_id': 1725, 'first_name': 'MARY', 'last_name': 'SMITH', 'amount': Decimal('4.99')}
{'rental_id': 2308, 'first_name': 'MARY', 'last_name': 'SMITH', 'amount': Decimal('4.99')}
{'rental_id': 2363, 'first_name': 'MARY', 'last_name': 'SMITH', 'amount': Decimal('0.99')}
{'rental_id': 3284, 'first_name': 'MARY', 'last_name': 'SMITH', 'amount': Decimal('3.99')}
{'rental_id': 4526, 'first_name': 'MARY', 'last_name': 'SMITH', 'amount': Decimal('5.99')}


In [91]:
# Cross Join (Cartesian Product)
query = """
SELECT customer.first_name, customer.last_name, film.title
FROM customer
CROSS JOIN film
LIMIT 10;
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(row)


{'first_name': 'AUSTIN', 'last_name': 'CINTRON', 'title': 'ACADEMY DINOSAUR'}
{'first_name': 'WADE', 'last_name': 'DELVALLE', 'title': 'ACADEMY DINOSAUR'}
{'first_name': 'FREDDIE', 'last_name': 'DUGGAN', 'title': 'ACADEMY DINOSAUR'}
{'first_name': 'ENRIQUE', 'last_name': 'FORSYTHE', 'title': 'ACADEMY DINOSAUR'}
{'first_name': 'TERRENCE', 'last_name': 'GUNDERSON', 'title': 'ACADEMY DINOSAUR'}
{'first_name': 'EDUARDO', 'last_name': 'HIATT', 'title': 'ACADEMY DINOSAUR'}
{'first_name': 'RENE', 'last_name': 'MCALISTER', 'title': 'ACADEMY DINOSAUR'}
{'first_name': 'TERRANCE', 'last_name': 'ROUSH', 'title': 'ACADEMY DINOSAUR'}
{'first_name': 'KENT', 'last_name': 'ARSENAULT', 'title': 'ACADEMY DINOSAUR'}
{'first_name': 'SETH', 'last_name': 'HANNON', 'title': 'ACADEMY DINOSAUR'}


In [93]:
# Performing Math Across Joined Columns (Total Revenue Per Customer)
query = """
SELECT customer.customer_id, customer.first_name, customer.last_name,
       SUM(payment.amount) AS total_spent
FROM customer
JOIN payment ON customer.customer_id = payment.customer_id
GROUP BY customer.customer_id
ORDER BY total_spent DESC
LIMIT 10;
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(row)


{'customer_id': 526, 'first_name': 'KARL', 'last_name': 'SEAL', 'total_spent': Decimal('221.55')}
{'customer_id': 148, 'first_name': 'ELEANOR', 'last_name': 'HUNT', 'total_spent': Decimal('216.54')}
{'customer_id': 144, 'first_name': 'CLARA', 'last_name': 'SHAW', 'total_spent': Decimal('195.58')}
{'customer_id': 137, 'first_name': 'RHONDA', 'last_name': 'KENNEDY', 'total_spent': Decimal('194.61')}
{'customer_id': 178, 'first_name': 'MARION', 'last_name': 'SNYDER', 'total_spent': Decimal('194.61')}
{'customer_id': 459, 'first_name': 'TOMMY', 'last_name': 'COLLAZO', 'total_spent': Decimal('186.62')}
{'customer_id': 469, 'first_name': 'WESLEY', 'last_name': 'BULL', 'total_spent': Decimal('177.60')}
{'customer_id': 468, 'first_name': 'TIM', 'last_name': 'CARY', 'total_spent': Decimal('175.61')}
{'customer_id': 236, 'first_name': 'MARCIA', 'last_name': 'DEAN', 'total_spent': Decimal('175.58')}
{'customer_id': 181, 'first_name': 'ANA', 'last_name': 'BRADLEY', 'total_spent': Decimal('174.66')

In [97]:
# Using IS NULL to Find Missing Data (Customers with No Payments)
query = """
SELECT customer.customer_id, customer.first_name, customer.last_name
FROM customer
LEFT JOIN payment ON customer.customer_id = payment.customer_id
WHERE payment.payment_id IS NULL
LIMIT 10;
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(row)


In [101]:
# Bonus: Math on Columns from Joined Tables (Example: Payment Percent Change Year-over-Year)
query = """
WITH payments_by_year AS (
    SELECT customer_id, YEAR(payment_date) AS year, SUM(amount) AS yearly_payment
    FROM payment
    GROUP BY customer_id, year
)
SELECT p1.customer_id, p1.year AS year_2005, p1.yearly_payment AS payment_2005,
       p2.year AS year_2006, p2.yearly_payment AS payment_2006,
       ROUND((p2.yearly_payment - p1.yearly_payment) / p1.yearly_payment * 100, 2) AS percent_change
FROM payments_by_year p1
JOIN payments_by_year p2
ON p1.customer_id = p2.customer_id
AND p1.year = 2005
AND p2.year = 2006;
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(row)


{'customer_id': 5, 'year_2005': 2005, 'payment_2005': Decimal('143.63'), 'year_2006': 2006, 'payment_2006': Decimal('0.99'), 'percent_change': Decimal('-99.31')}
{'customer_id': 9, 'year_2005': 2005, 'payment_2005': Decimal('84.78'), 'year_2006': 2006, 'payment_2006': Decimal('4.99'), 'percent_change': Decimal('-94.11')}
{'customer_id': 11, 'year_2005': 2005, 'payment_2005': Decimal('105.77'), 'year_2006': 2006, 'payment_2006': Decimal('0.99'), 'percent_change': Decimal('-99.06')}
{'customer_id': 14, 'year_2005': 2005, 'payment_2005': Decimal('112.73'), 'year_2006': 2006, 'payment_2006': Decimal('4.99'), 'percent_change': Decimal('-95.57')}
{'customer_id': 15, 'year_2005': 2005, 'payment_2005': Decimal('130.70'), 'year_2006': 2006, 'payment_2006': Decimal('3.98'), 'percent_change': Decimal('-96.95')}
{'customer_id': 21, 'year_2005': 2005, 'payment_2005': Decimal('152.66'), 'year_2006': 2006, 'payment_2006': Decimal('2.99'), 'percent_change': Decimal('-98.04')}
{'customer_id': 22, 'year

# Chapter 7

Chapter 7 queries are performed in SQLite in the notebook

In [107]:
# First - Connect to SQLite ( sakila.db exists locally)
import sqlite3

conn = sqlite3.connect('sakila.db')
cursor = conn.cursor()


In [115]:
# Primary Key & Foreign Key Check (PRAGMA)
# To inspect the structure of the payment table:

query = "PRAGMA table_info(payment)"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)


In [117]:
# Add UNIQUE Constraint (Email in Customer Table)
# SQLite only supports adding unique constraints when the table is first created. If modifying an existing table, you need to rebuild the table.
query = """
CREATE TABLE IF NOT EXISTS customer_new (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT UNIQUE,
    active INTEGER
)
"""
cursor.execute(query)
print("New customer table with UNIQUE email created.")


New customer table with UNIQUE email created.


In [119]:
# Add NOT NULL Constraint (on amount in payment)
# Again, you can't directly ALTER COLUMN in SQLite — you need to rebuild the table if schema changes are required.

# Example for new payment table:
query = """
CREATE TABLE IF NOT EXISTS payment_new (
    payment_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    amount REAL NOT NULL,
    payment_date TEXT,
    FOREIGN KEY(customer_id) REFERENCES customer(customer_id)
)
"""
cursor.execute(query)
print("New payment table with NOT NULL on amount created.")


New payment table with NOT NULL on amount created.


In [121]:
# CHECK Constraint (e.g., enforce active in customer table is 0 or 1)
# In SQLite:
query = """
CREATE TABLE IF NOT EXISTS customer_checked (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    active INTEGER CHECK(active IN (0, 1))
)
"""
cursor.execute(query)
print("New customer table with CHECK constraint on active created.")


New customer table with CHECK constraint on active created.


In [123]:
# Composite Primary Key Example (Inventory-Rental Link Table)
query = """
CREATE TABLE IF NOT EXISTS inventory_rental_link (
    inventory_id INTEGER,
    rental_id INTEGER,
    PRIMARY KEY (inventory_id, rental_id)
)
"""
cursor.execute(query)
print("Composite Primary Key table created.")


Composite Primary Key table created.


In [133]:
# Aggregate Functions (SUM, AVG - Total and Average Payment)
query = """
INSERT INTO payment_new (payment_id, customer_id, amount, payment_date)
VALUES
    (1, 1, 5.99, '2024-02-28'),
    (2, 2, 2.99, '2024-02-28'),
    (3, 3, 7.50, '2024-02-28')
"""
cursor.execute(query)
conn.commit()
print("Sample data inserted into payment_new")

# Aggregate Functions (SUM, AVG - Total and Average Payment)
query = """
SELECT SUM(amount) AS total_revenue, AVG(amount) AS avg_payment
FROM payment_new
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(f"Total Revenue: ${row[0]:.2f}, Average Payment: ${row[1]:.2f}")


Sample data inserted into payment_new
Total Revenue: $16.48, Average Payment: $5.49


In [139]:
# Find Missing Foreign Key Matches (Customers with No Payments)

query = """
SELECT customer_checked.customer_id, customer_checked.first_name, customer_checked.last_name
FROM customer_checked
LEFT JOIN payment_new ON customer_checked.customer_id = payment_new.customer_id
WHERE payment_new.payment_id IS NULL
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(row)


In [143]:
# Index Creation (Index on last_name)
query = """
CREATE INDEX IF NOT EXISTS idx_last_name ON customer_checked(last_name)
"""
cursor.execute(query)
print("Index created on customer.last_name")


Index created on customer.last_name


In [145]:
# Query Performance Check (EXPLAIN Equivalent)
query = """
EXPLAIN QUERY PLAN SELECT * FROM customer_checked WHERE last_name = 'SMITH'
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(row)


(3, 0, 0, 'SEARCH customer_checked USING INDEX idx_last_name (last_name=?)')


Since SQLite is a lighter database:

- Constraints like CHECK are supported but enforcement is not as strict.
- Schema alterations require table rebuilds (like the UNIQUE and NOT NULL examples).
